<a href="https://colab.research.google.com/github/wileyw/DeepLearningDemos/blob/master/TabularXGBoost/TabularDataXGBoostTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toy tutorial (https://www.datacamp.com/tutorial/xgboost-in-python)

In [8]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [9]:
print(boston.data.shape)

(506, 13)


In [10]:
print(boston.feature_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [11]:
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [12]:
import pandas as pd

data = pd.DataFrame(boston.data)
data.columns = boston.feature_names

In [13]:
data['PRICE'] = boston.target

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  PRICE    506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [15]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [17]:
X, y = data.iloc[:,:-1],data.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [20]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [21]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

[03:12:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [22]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 10.449300


# XGBoost Eye Movement Dataset

In [3]:
# Download eye movements dataset (https://www.openml.org/search?type=data&sort=runs&id=1044&status=active )
!wget https://www.openml.org/data/download/53927/eye_movements.arff

--2022-08-16 02:56:39--  https://www.openml.org/data/download/53927/eye_movements.arff
Resolving www.openml.org (www.openml.org)... 131.155.11.11
Connecting to www.openml.org (www.openml.org)|131.155.11.11|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://api.openml.org/data/download/53927/eye_movements.arff [following]
--2022-08-16 02:56:40--  https://api.openml.org/data/download/53927/eye_movements.arff
Resolving api.openml.org (api.openml.org)... 131.155.11.11
Connecting to api.openml.org (api.openml.org)|131.155.11.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1380106 (1.3M) [text/plain]
Saving to: ‘eye_movements.arff.1’

eye_movements.arff. 100%[===================>]   1.32M  2.51MB/s    in 0.5s    

2022-08-16 02:56:41 (2.51 MB/s) - ‘eye_movements.arff.1’ saved [1380106/1380106]

eye_movements.arff  eye_movements.arff.1  sample_data


In [6]:
# Load data
from scipy.io import arff
import pandas as pd
data = arff.loadarff('eye_movements.arff')
df = pd.DataFrame(data[0])

In [7]:
df.head()

,lineNo,assgNo,fixcount,firstPassCnt,P1stFixation,P2stFixation,prevFixDur,firstfixDur,firstPassFixDur,nextFixDur,...,regressLen,nextWordRegress,regressDur,pupilDiamMax,pupilDiamLag,timePrtctg,nWordsInTitle,titleNo,wordNo,label
0,1.0,1.0,1.0,1.0,b'1',b'0',0.0,100.0,100.0,99.0,...,0.0,b'0',0.0,0.0095,0.145,0.0131,7.0,4.0,3.0,b'0'
1,2.0,1.0,1.0,1.0,b'1',b'0',99.0,278.0,278.0,159.0,...,0.0,b'0',0.0,0.0095,0.183,0.0363,7.0,1.0,1.0,b'0'
2,3.0,1.0,1.0,1.0,b'1',b'0',278.0,159.0,159.0,159.0,...,0.0,b'0',0.0,0.0370,0.183,0.0208,7.0,1.0,3.0,b'0'
3,4.0,1.0,1.0,1.0,b'1',b'0',159.0,159.0,159.0,139.0,...,0.0,b'0',0.0,0.0370,0.183,0.0208,7.0,1.0,5.0,b'0'
4,5.0,1.0,1.0,1.0,b'1',b'0',159.0,139.0,139.0,239.0,...,0.0,b'0',0.0,0.0390,0.183,0.0182,7.0,1.0,6.0,b'0'


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np